In [ ]:
# import packages
import pandas as pd
import numpy as np
import re

In [ ]:
#create dataframes

course_path = 'C:/Users/ywa/Documents/Data/Moodle Query/Nov2025/Course - Category - Enrollments.csv' 
date_path = 'C:/Users/ywa/Documents/Data/Moodle Query/Nov2025/Course - First Enrollment.csv'

course_df = pd.read_csv(course_path)
date_df = pd.read_csv(date_path)


#cert_info.head()
date_df.head()

In [ ]:
# drop firstEnrollment col of course_df

course_df = course_df.drop('firstEnrollment', axis = 1)

course_df.head()

## Top 5 new comers

In [ ]:
# merge course_df and date_df

print(len(course_df))
print(len(date_df))

best_df = course_df.merge(date_df, on = 'courseID', how = 'left')

print(len(course_df))
print(len(date_df))

print(len(best_df))

best_df.head()


In [ ]:
# check missing values

print(best_df.isna().sum())

best_df = best_df.dropna()

print(best_df.isna().sum())


In [ ]:
# filter visible and live courses

best_df = best_df[(best_df['visible'] == 1) & (best_df['enrollments'] > 10)]

best_df


In [ ]:
# keep only the y-m-d string

date_regex = r'^(\d{4}-\d{2}-\d{2})'

best_df['firstEnrollment'] = best_df['firstEnrollment'].str.extract(date_regex)

best_df.head()



In [ ]:
print(best_df.info())
best_df['firstEnrollment'] = pd.to_datetime(best_df['firstEnrollment'])

print(best_df.info())

In [ ]:
# new comers top 5

# filter the courses released in the current year

current_y = 2025

# filter based on the current_y

mask_y = best_df['firstEnrollment'].dt.year == current_y

filtered_y = best_df[mask_y]

filtered_y.head()

# sort values based on enrollements

new_comers = filtered_y.sort_values(by = 'enrollments', ascending = False)

new_comers.head(6)


## Top 5 all time favourites

In [ ]:
# HPI should be also involved.
hpi_path = 'C:/Users/ywa/Documents/Data/hpi/2024-12-31_OverallCourseSummaryReport.csv'

hpi_df = pd.read_csv(hpi_path)

hpi_df.head()

hpi_df.columns

In [ ]:
# select the course-enrollment subset

hpi_df = hpi_df[['id', 'title', 'enrollments_total']]

hpi_df.head()


In [ ]:
# get rid of the missing values

hpi_df = hpi_df.dropna()

hpi_df.isnull().sum()

In [ ]:
# remove the "" in the title --> for "merging table"

hpi_df['title'] = hpi_df['title'].str.strip('"')


In [ ]:
# sort values of the hpi enrolment
hpi_df = hpi_df.sort_values(by = 'enrollments_total', ascending = False)

print(hpi_df.shape)
hpi_df.head(50)


In [ ]:
# inspect the top 50 of the moodle enrolment
print(best_df.shape)

best_df.head(50)


In [ ]:
# perform rapidfuzz the best_df  and hpi_df

#!pip install rapidfuzz

from rapidfuzz import process, fuzz

choices = hpi_df['title'].tolist()

matches = best_df['courseName'].apply(\
                              lambda x : process.extractOne(\
                                                           x, choices, scorer = fuzz.WRatio) )

best_df['best_title'] = [m[0] if m is not None else None for m in matches]
best_df['score'] =[m[1] if m is not None else None for m in matches]

threshold = 95

best_df_matched = best_df[best_df['score'] >= threshold].copy()

result = best_df_matched.merge(hpi_df, left_on = 'best_title', right_on = 'title', how = 'left')


result.shape


In [ ]:
result['enrol'] = result['enrollments'] + result['enrollments_total']

result_sorted = result.sort_values(by = 'enrol', ascending = False)


# All time favouriates top 5

all_time_favouriates = result_sorted.head(6)

all_time_favouriates

In [ ]:
# drop certain cols and rows

all_time_favouriates = all_time_favouriates.drop(['firstEnrollment', 'id'], axis = 1)

all_time_favouriates = all_time_favouriates.drop(index = 39)

all_time_favouriates

In [ ]:
# comparing with moodle table
best5_moodle = best_df.head(5)

# print the max and min values to make sure the correctness
min_moodle = best5_moodle['enrollments'].min()

max_moodle = best5_moodle['enrollments'].max()

print(min_moodle, max_moodle)

min_all_time = all_time_favouriates['enrol'].min()

max_all_time = all_time_favouriates['enrol'].max()

print(min_all_time, max_all_time)


In [ ]:
## For AutoML/FoAI 1 needs to be re-checked just in case. 

## Top 5 enrolment of the year

In [ ]:
# import the enrolment table

enrol_path = 'C:/Users/ywa/Documents/Data/Moodle Query/Nov2025/TotalCourseEnrollments.csv'

enrol_df = pd.read_csv(enrol_path)

print(enrol_df.shape)

enrol_df.head()

In [ ]:
# check timestamp (sanity check)

enrol_df.info()

regex_year = r'(\d{4})'

enrol_df['TimeStamp'] = enrol_df['Time created'].str.extract(regex_year).astype(int)

print((enrol_df['TimeStamp'] < 2025).sum())

In [ ]:
# count the courses

course_enrol_count = pd.DataFrame(enrol_df.value_counts('Course full name'))

course_enrol_count = course_enrol_count.reset_index()

# rename the columns
course_enrol_count = course_enrol_count.rename(columns = {'Course full name' : 'Course', 0 : 'Enrolment'})


In [ ]:
course_enrol_count.sort_values(by = 'Enrolment', ascending = False).head()